In [ ]:
import numpy as np
import pandas as pd

import fileio, filtering, kmers, analysis

# Read and filter all files

In [ ]:
allSeqs = fileio.readData("./data/")

allSeqs_filtered = filtering.filter_by_coverage(allSeqs, threshold=0.7)

# Define settings and preprocess data

In [ ]:
m = 255     # number of permutations
k = 8       # kmer size

In [ ]:
allKmers = {exp: kmers.toKmers(df, kmer_size=k) for exp, df in allSeqs_filtered.items()}

# weighting of kmers
df = pd.DataFrame.from_dict(allKmers, orient="columns").fillna(0)
kmermatrix = np.array([df[exp] for exp in df.columns])

# Weighted Minhashing

In [ ]:
from datasketch import WeightedMinHashGenerator

n_repetitions = 30
allSignatures_byrun = [None]*n_repetitions
allBinarySignatures_byrun = [None]*n_repetitions

for i in range(n_repetitions):
    wmg = WeightedMinHashGenerator(kmermatrix.shape[1], sample_size=m, seed=i)
    allSignatures_byrun[i] = {exp: np.mod(w.digest()[:,0], 256) for exp, w in zip(allKmers.keys(), wmg.minhash_many(kmermatrix))}
    allBinarySignatures_byrun[i] = {exp: ''.join([bin(s)[2:].zfill(8) for s in sig]) for exp, sig in allSignatures_byrun[i].items()}

In [ ]:
woverlaps_byrun = np.zeros((n_repetitions, 39, 39))
for i, rep in enumerate(allSignatures_byrun):
    woverlaps_byrun[i] = np.array([[sum(np.array(s1) == np.array(s2))/m for s1 in rep.values()] for s2 in rep.values()])

woverlaps = np.mean(woverlaps_byrun, axis=0)

In [ ]:
hammings_byrun = np.zeros((n_repetitions, 39, 39))

for i, rep in enumerate(allBinarySignatures_byrun):
    hammings_byrun[i] = np.array([[sum([ord(a) ^ ord(b) for a, b in zip(s1, s2)])/len(s1) for s1 in rep.values()] for s2 in rep.values()])

hammings = np.mean(hammings_byrun, axis=0)

# Fuzzy Extractor

In [ ]:
from reedsolo import RSCodec, ReedSolomonError


# A fuzzy extractor via the code offset construction
class fuzzy_extractor:

    def __init__(self, r, n):
        # int r: number of error correcting symbols
        # int n: number of information symbols
        self.r = r
        self.rsc = RSCodec(r)
        self.n = n   

    def bytearraysubstraction(self, a, b):
        res = bytearray(len(a))
        for i in range(len(a)):
            res[i] = ( a[i] - b[i] ) % 256
        return res 
    
    def encode(self, w):
        w = bytearray(w)
        # choose a codeword uniformly at random        
        inf = bytearray(np.random.randint(256, size = self.n-self.r).tolist())
        c = self.rsc.encode(inf)
        if len(c) != self.n:
            raise Exception('RSCodec unexpeced behavior')
        # h = w - c
        h = self.bytearraysubstraction(w, c)
        return inf, h
    
    def decode(self, wdash, h):
        wdash = bytearray(wdash)
        # c = w - h
        diff = self.bytearraysubstraction(wdash, h)
        try:
            decoded_msg, _, _ = self.rsc.decode(diff)
        except ReedSolomonError as e:
            decoded_msg = None
        
        return decoded_msg


def test_fuzzy_extractor(fuz_ext, seq1, seq2):
    w = bytearray(seq1)
    wdash = bytearray(seq2)

    key, helper = fuz_ext.encode(w)
    key_dash = fuz_ext.decode(wdash, helper)
    return key == key_dash

In [ ]:
fuz_ext = fuzzy_extractor(r=m-32, n=m) # r = redundancy, n = length of the codeword
key_byrun = np.zeros((n_repetitions, 39, 39))

for i, rep in enumerate(allSignatures_byrun):
    key_byrun[i] = np.array([[test_fuzzy_extractor(fuz_ext, list(s1), list(s2)) for s1 in rep.values()] for s2 in rep.values()])

key = np.mean(key_byrun, axis=0, dtype=float)

# Save data

In [ ]:
np.savetxt("./results_analysis/minhash_overlaps.csv", woverlaps, delimiter=",")
np.savetxt("./results_analysis/minhash_hamming.csv", hammings, delimiter=",")
np.savetxt("./results_analysis/minhash_fuzzykeys.csv", key, delimiter=",")

# All datasets

In [ ]:
analysis.plot_matrix(woverlaps).show()

positive_matrix, negative_matrix = analysis.generate_truthtables()
fig = analysis.plot_hists(woverlaps, positive_matrix, negative_matrix)
fig.show()
fig.write_image("./results_analysis/unconstrained_minhash_overlaps.svg")

In [ ]:
analysis.plot_matrix(hammings).show()

positive_matrix, negative_matrix = analysis.generate_truthtables()
fig = analysis.plot_hists(hammings, positive_matrix, negative_matrix)
fig.show()
fig.write_image("./results_analysis/unconstrained_minhash_hamming.svg")

In [ ]:
analysis.plot_matrix(key).show()

positive_matrix, negative_matrix = analysis.generate_truthtables()
fig = analysis.plot_hists(key, positive_matrix, negative_matrix)
fig.show()
fig.write_image("./results_analysis/unconstrained_fuzzy_keys.svg")

# Constrained dataets

In [ ]:
analysis.plot_matrix(woverlaps).show()

to_exclude = set([16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 33, 34, 37, 38])
positive_matrix, negative_matrix = analysis.generate_truthtables(to_exclude)
fig = analysis.plot_hists(woverlaps, positive_matrix, negative_matrix)
fig.show()
fig.write_image("./results_analysis/constrained_minhash_overlaps.svg")

In [ ]:
analysis.plot_matrix(hammings).show()

to_exclude = set([16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 33, 34, 37, 38])
positive_matrix, negative_matrix = analysis.generate_truthtables(to_exclude)
fig = analysis.plot_hists(hammings, positive_matrix, negative_matrix)
fig.show()
fig.write_image("./results_analysis/constrained_minhash_hamming.svg")

In [ ]:
analysis.plot_matrix(key).show()

to_exclude = set([16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 33, 34, 37, 38])
positive_matrix, negative_matrix = analysis.generate_truthtables(to_exclude)
fig = analysis.plot_hists(key, positive_matrix, negative_matrix)
fig.show()
fig.write_image("./results_analysis/constrained_fuzzy_keys.svg")